In [4]:
import sys
sys.path.append("./git/")

# The model

Suppose you are trying to predict some value $y_i$ using some variable $x_i$, and you use a linear model to try to do so.

Taking the basic equation of a straight line, you suppose that the gradient and y-axis cutting point are $\alpha$ and $\beta$, giving you:

$$y_i = \beta x_i + \alpha + \varepsilon_i$$

where $\varepsilon_i$ is some small error term representing the fact that there are other factors which this model does not take into account.

Assuming we've already figured out $\alpha$ and $\beta$, then making prediction is trivially easy:

In [1]:
def predict(alpha:float, 
            beta:float, 
            x_i:float) -> float:
    return beta*x_i + alpha

But how do you pick your `alpha` and `beta` values? Since we already know some actual values of y_i, you can start by computing the error for each pair of values i.e. how wrong are the predictions for each pair of alpha and beta?

In [2]:
def error(alpha: float, 
          beta: float,
          x_i: float,
          y_i:float) -> float:
    """
    The error from predicting beta * x_i + alpha
    when the actual value is y_i.
    """
    return predict(alpha,beta,x_i) - y_i

This function gives us the error value of each point, but what about some value of error for the whole dataset?

You can't just add up the individual errors of each data point because + and - errors will cancel out. That's why you use the *mean squared error*:

In [5]:
from scratch.linear_algebra import Vector

def sum_of_sqerrors(alpha: float,
                    beta: float,
                    x: Vector,
                    y: Vector) -> float:
    return sum(error(alpha,beta,x_i,y_i)**2 
               for x_i, y_i in zip(x,y))

The *least squares solution* is the pair of alpha-beta values that minimise the sum of squared errors.

In [7]:
from typing import Tuple
from scratch.linear_algebra import Vector
from scratch.statistics import correlation, standard_deviation, mean

def least_squares_fit(x: Vector, 
                      y: Vector) -> Tuple[float,float]:
    """
    Given two vectors x and y,
    find the least-squares values of alpha and beta
    """
    beta = correlation(x,y) * standard_deviation(y) / standard_deviation(x)
    alpha = mean(y) - beta*mean(x)
    return alpha, beta

Ignoring the algebra and calculus behind this equation, lets think about the intuition behind it.

Our chosen alpha value says that when we see the average value of the independent variable `x` we predict the average value of variable `y`.

Our chosen beta value means that when the input value increases by `standard_deviation(x)`, the prediction increases by `correlation(x,y) * standard_deviation(y)`.

In the case where `x` and `y` are perfectly correlated, one standard deviation increase in `x` results in one standard deviation increase in `y`, our prediction.

When they are perfectly anti-correlated, one standard deviation increase in `x` results in a one standard deviation *decrease* in `y`, our prediction.

When the correlation is 0, beta is 0, which means that changes in `x` don't affect the prediction at all.

Let's test this:

In [8]:
x = [i for i in range(-100,100,10)]
y = [3*i -5 for i in x]

assert least_squares_fit(x,y) == (-5,3)

In [9]:
from scratch.statistics import num_friends_good, daily_minutes_good

alpha, beta = least_squares_fit(num_friends_good,daily_minutes_good)
assert 22.9 < alpha < 23.0
assert 0.9 < beta < 0.905

In [10]:
print(alpha)
print(beta)

22.94755241346903
0.903865945605865


This means that we would expect: 
- a user with n friends to spend 22.95 + (n x 0.903) minutes on the site each day, in this particular example;
- a user with no friends (x) to still spend about 23 minutes a day on the site
- for each additional friend, we expect a user to spend almost a minute more on the site each day.

We can test and understand this intuition by looking a chart of the scattered data points with the straight line regression superimposed, but that's not very scientific.

The scientific way to do it is to use the 'r-squared value' or the 'coefficient of determination', which measures the fraction of the total variation in the dependent variable that is captured by the model:

In [11]:
from scratch.statistics import de_mean

def total_sum_of_squares(y: Vector) -> float:
    """
    the total squared variation of y_i's from their mean
    """
    return sum(v**2 for v in de_mean(y))

def r_squared(alpha: float, 
              beta:float, 
              x: Vector, 
              y: Vector) -> float:
    return 1.0 - (sum_of_sqerrors(alpha,beta,x,y)/total_sum_of_squares(y))

rsq = r_squared(alpha,
                beta,
                num_friends_good,
                daily_minutes_good)
assert 0.328 < rsq < 0.330

The higher the r-squared value, the better our model fits the data. 

In this case, the value is 0.329 which tells us that the model is only kind of meh at fitting the data, and that clearly there are other factors at play.

# Gradient descent

If we write `theta = [alpha, beta]` we can also solve this using gradient descent:

In [12]:
import random
import tqdm
from scratch.gradient_descent import gradient_step

num_epochs = 10000
random.seed(0)
guess = [random.random(), random.random()]
learning_rate = 0.00001

with tqdm.trange(num_epochs) as t:
    for _ in t:
        alpha, beta = guess
        grad_a = sum(2*error(alpha,beta,x_i,y_i) 
                     for x_i, y_i in zip(num_friends_good,
                                         daily_minutes_good))
        grad_b = sum(2*error(alpha,beta,x_i,y_i) * x_i 
                     for x_i,y_i in zip(num_friends_good,
                                        daily_minutes_good))
        
        loss = sum_of_sqerrors(alpha,
                               beta,
                               num_friends_good,
                               daily_minutes_good)
        
        t.set_description(f"loss: {loss:.3f}")
        
        guess = gradient_step(guess,
                              [grad_a, grad_b],
                              -learning_rate)
        
alpha,beta = guess
assert 22.9 < alpha < 23.0
assert 0.9 < beta < 0.905

loss: 13196.619: 100%|██████████████████████████████████████████████████████████| 10000/10000 [00:10<00:00, 976.55it/s]


In [13]:
print(alpha)
print(beta)

22.947552155340915
0.9038659662765034


# Maximum likelihood estimation

Why choose least squares as our method of assessing errors?

One argument is from *maximum likelihood estimation*. Imagine we have a sample of data $v_1,...,v_n$ that comes from a distribution that depends on some unknown parameter $\theta$:

$$p(v_1,...v_n|\theta)$$

If we didn't know $\theta$ we could turn around and think of this quantity as the likelihood of  $\theta$ given the sample:

$$L(\theta|v_1,...,v_n)$$

Under with approach, the most likely $\theta$ is the value that maximises this likelihood function - that is, the value that makes the observed data the most probable. Given a continuous distribution where we have a probability density function rather than a mass function, we can do the same thing.

One assumption that's made about simple linear regression models is that the regression errors are normally distributed with mean 0 and some (known) standard deviation $\sigma$. If thats the case, then the likelihood based on seeing a pair `(x_i, y_i)` is:

$$L(\alpha,\beta|x_i,y_i,\sigma) = (1/\sqrt(2\pi\sigma))exp(-(y_i - \alpha - \beta x_i)^2/(2\sigma^2))$$

The likelihood based on the entire dataset is the product of the individual likelihoods, which is largest precisely when `alpha` and `beta` are chosen to minimise the sum of squared errors.

Therefore, in this case, under these assumptions, minimising the sum of squared errors is equivalent to maximising the likelihood of the observed data.